# Peer graded Assignment - Week 3
## Segmenting and Clustering Neighborhoods in Toronto
#### Friday, 8th January 2021

Hello and welcome to my notebook for the assignment for Week 3. 

## A - Scraping from Wikipedia page
We will capture data from the wikipedia page mentionned (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to create a pandas dataframe with PostalCode, Borough, and Neighborhood.

### I - Extraction from the url page

In [1]:
#First scraping from the url 
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

#Print the number of dataframe detected
print(len(dfs))

3


In [2]:
#Verify that the first one is the one we need
print(dfs[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

In [3]:
#Define our final df
df=dfs[0]

#Read the first five row of our df
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### II - Cleaning of the dataframe
The main goal here is to: 
- Ignore cells with a borough that is Not assigned.
- If a cell has a Not assigned  neighborhood but a borough, put the same name of the borough.
- Check if the df has more than one neighborhood in one postal code area. If it is the case, gather the rows per one postal code.

After cleaning the dataframe, we will use the shape function to see the result.


In [4]:
#Ignore cells with a borough that is Not assigned.
#First, Let's see where "Not assigned" values are in the Borough column
if 'Not assigned' in df['Borough'].values : 
    print("\nThis value exists in Dataframe, column Borough") 
else : 
    print("\nThis value does not exists in Dataframe, column Borough") 


This value exists in Dataframe, column Borough


In [5]:
#Let's create a new df withouth these values
df=df[df.Borough !='Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
#Check of the df
if 'Not assigned' in df['Borough'].values : 
    print("\nThis value exists in Dataframe, column Borough") 
else : 
    print("\nThis value does not exists in Dataframe, column Borough") 


This value does not exists in Dataframe, column Borough


In [7]:
#If a cell has a Not assigned neighborhood but a borough, put the same name of the borough.
#First, Let see where "Not assigned" values are in the Neighbourhood column
if 'Not assigned' in df['Neighbourhood'].values : 
    print("\nThis value exists in Dataframe, column Neighbourhood") 
else : 
    print("\nThis value does not exists in Dataframe, column Neighbourhood") 


#The value does not seem to exist ! 


This value does not exists in Dataframe, column Neighbourhood


In [8]:
#Check if the df has more than one neighborhood in one postal code area.
count_check=df['Postal Code'].value_counts()

In [9]:
for i in count_check:
    if i>1:
        print('there is an ocurrence in the line '+ count_check.index)
    else:
        continue
print("Finish !")

Finish !


In [10]:
#Print the shape of the df
df.shape

(103, 3)

In [11]:
df=df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### III - Adding Latitude and longitude
We will add latitude and longitude to our dataframe for each postal code.
To do that, we will import the needed data from another CSV File (http://cocl.us/Geospatial_data)

In [12]:
#First, let's import our csv
latlon=pd.read_csv('Geospatial_Coordinates.csv')
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Add the latitude and the longitude column by merging the two dataframes
new_df=pd.merge(df,latlon,how='left',on='Postal Code')
new_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
#Verify the shape of the new df
new_df.shape

(103, 5)

### IV - Clustering analysis

Here, we will perform a clustering analysis using k-means to gathers the relevant point in the neighbourhoods.

In [15]:
#import the needed libraries

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



#### A - Get the venues of a each neighborhood
Let's create a new dataframe only with Boroughs that contains the word "Toronto".

In [16]:
toronto_data= new_df[new_df['Borough'].str.contains('Toronto', case=False)].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


First map without clustering to have a big picture of the borough "Downtown Toronto".

Use Foursquare.

In [17]:
CLIENT_ID = 'IK0OCJP2ALP4KSMAD3CT3XO1O5FQRF1ME25022XTD3IT4SSP' # your Foursquare ID
CLIENT_SECRET = 'BMVBZVO0MNJLW2HHSCWAKLXOZYSLXAZUAHPUBWP5GCZCUKFG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: IK0OCJP2ALP4KSMAD3CT3XO1O5FQRF1ME25022XTD3IT4SSP
CLIENT_SECRET:BMVBZVO0MNJLW2HHSCWAKLXOZYSLXAZUAHPUBWP5GCZCUKFG


Create a function to capture all the venues for each neighborhood.

In [20]:
def getNearbyVenues (names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create an API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        #return the relevant informations from this request
        venues_list.append([(name, lat, lng, 
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],  
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [21]:
toronto_venues=getNearbyVenues(names=toronto_data['Neighbourhood'], latitudes = toronto_data['Latitude'], longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [22]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [23]:
toronto_venues.shape

(1618, 7)

#### B - One hot encoding

In [24]:
#one hot encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

#add the neighbourhood column back to the dataframe
toronto_onehot['Neighbourhood']= toronto_venues['Neighbourhood']

# move neighborhood column to the first column
final_columns= [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[final_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(1618, 235)

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.0625,0.125,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.016949,0.016949
5,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012821,0.000000,0.00,...,0.012821,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.000000,0.025641
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
toronto_grouped.shape

(39, 235)

In [39]:
#Let's create a new dataframe only with the Neighbourhood for the next step
toronto_grouped_fin= toronto_grouped[['Neighbourhood']]
toronto_grouped_fin.head()

,Neighbourhood
0,Berczy Park
1,"Brockton, Parkdale Village, Exhibition Place"
2,"Business reply mail Processing Centre, South C..."
3,"CN Tower, King and Spadina, Railway Lands, Har..."
4,Central Bay Street


#### C - Clustering the venues for the downtown toronto neighbourhood

Set the number of clustering we want to have. 

In [40]:
#Set the number of cluster
kcluster = 4

#Remove the first column for the clustering
toronto_grouped_cl=toronto_grouped.drop('Neighbourhood', 1)

#run k-means clustering
kmeans=KMeans(n_clusters=kcluster, random_state=0).fit(toronto_grouped_cl)

#check the clusters generated
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Add the columnn with the cluster 

In [41]:
# add clustering labels
toronto_grouped_fin.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped_fin.head()

,Cluster Labels,Neighbourhood
0,0,Berczy Park
1,0,"Brockton, Parkdale Village, Exhibition Place"
2,0,"Business reply mail Processing Centre, South C..."
3,0,"CN Tower, King and Spadina, Railway Lands, Har..."
4,0,Central Bay Street


In [42]:
toronto_merged = toronto_grouped_fin

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_data.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
1,0,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,43.636847,-79.428191
2,0,"Business reply mail Processing Centre, South C...",M7Y,East Toronto,43.662744,-79.321558
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,Downtown Toronto,43.628947,-79.394420
4,0,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383


#### D - Final map of the clusters

In [45]:
import numpy as np

In [47]:
#Create the map
map_clusters = folium.Map(location=[43.7001100, -79.4163000], zoom_start=11)

#set color scheme for the clusters
x= np.arange (kcluster)
ys= [i + x + (i*x)**2 for i in range (kcluster)]
colors_array= cm.rainbow (np.linspace (0,1, len(ys)))
rainbow= [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors=[]

for lat, lon, poin, cluster in zip (toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poin) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters